In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import requests
from pathlib import Path
import json

In [ ]:
# Instagram post URL
POST_URL = ""  # PASTE YOUR INSTAGRAM POST URL HERE
DOWNLOAD_DIR = Path("instagram_downloads")
DOWNLOAD_DIR.mkdir(exist_ok=True)

In [ ]:
# Setup Chrome with mobile emulation and DevTools
options = webdriver.ChromeOptions()
options.add_argument('--auto-open-devtools-for-tabs')
options.add_experimental_option('mobileEmulation', {
    'deviceName': 'iPhone 12 Pro'
})
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(options=options)
print("Browser opened with DevTools enabled")

In [ ]:
# Navigate and refresh
driver.get(POST_URL)
time.sleep(3)
driver.refresh()
time.sleep(2)
print("Page loaded and refreshed")

In [ ]:
# Click next button until no more items
click_count = 0
while True:
    try:
        # Find next button (Instagram carousel)
        next_btn = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"], button[aria-label="next"]'))
        )
        next_btn.click()
        click_count += 1
        print(f"Clicked next: {click_count}")
        time.sleep(0.5)
    except (TimeoutException, NoSuchElementException):
        print("No more next button found")
        break

print(f"Total clicks: {click_count}")

In [ ]:
# Extract media URLs from network logs
logs = driver.get_log('performance')
media_urls = set()

for log in logs:
    try:
        message = json.loads(log['message'])['message']
        if message['method'] == 'Network.responseReceived':
            response = message['params']['response']
            url = response['url']
            mime_type = response.get('mimeType', '')
            
            # Capture images and videos from Network tab
            if 'image/' in mime_type or 'video/' in mime_type:
                if 'cdninstagram.com' in url:
                    media_urls.add(url)
    except:
        continue

print(f"Found {len(media_urls)} unique media items")
driver.quit()

In [ ]:
# Download all media (no duplicates)
for idx, url in enumerate(media_urls, 1):
    try:
        ext = '.jpg' if 'image/' in url or '.jpg' in url else '.mp4'
        filename = DOWNLOAD_DIR / f"item_{idx}{ext}"
        
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        filename.write_bytes(response.content)
        print(f"Downloaded {idx}/{len(media_urls)}: {filename.name}")
    except Exception as e:
        print(f"Failed {idx}: {e}")

print(f"\nAll downloads complete! Check '{DOWNLOAD_DIR}' folder")